In [36]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [37]:
import pandas as pd
import requests
import os
import sys
import datetime
import numpy as np
import re
import logging
from dateutil.relativedelta import relativedelta, TH
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
ohlcFl = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlcWithDelOI.csv"
yhfinanceFl = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\correctedEODDataYF.csv"

In [38]:
ohlc = pd.read_csv(ohlcFl)
ohlc["date"] = pd.to_datetime(ohlc.date).apply(lambda x: x.date())
yFin = pd.read_csv(yhfinanceFl)
yFin["Date"] = pd.to_datetime(yFin.Date, format="%Y%m%d").apply(lambda x: x.date())

In [53]:
ohlc.loc[ohlc.symbol == "NSE500", "date"].max()

datetime.date(2022, 3, 10)

In [39]:
yFin.rename(
    columns={
        "Date": "date",
        "Ticker": "symbol",
        "Close": "corrClose",
        "High": "corrHigh",
        "Low": "corrLow",
        "Open": "corrOpen",
        "Volume": "yfVolume",
    },
    inplace=True,
)

In [40]:
finalOHLC = pd.merge(
    left=yFin,
    right=ohlc,
    left_on=["symbol", "date"],
    right_on=["symbol", "date"],
    how="outer",
)

In [41]:
def getNifty500():

    nList = "nifty500"
    url = "https://archives.nseindia.com/content/indices/ind_" + nList + "list.csv"

    return pd.read_csv(url)


nifty500 = getNifty500()

In [42]:
tenYearAgo = datetime.date.today() - relativedelta(years=10)
finalOHLC10yr = finalOHLC.loc[finalOHLC.date >= tenYearAgo, :]

In [43]:
finalOHLC10yrNifty500 = finalOHLC10yr.loc[
    finalOHLC10yr.symbol.isin(nifty500.Symbol), :
].dropna(subset=["corrOpen", "corrClose", "corrHigh", "corrLow"])

In [44]:
finalOHLC10yrNifty500 = finalOHLC10yrNifty500[
    [
        "date",
        "symbol",
        "corrClose",
        "corrHigh",
        "corrLow",
        "corrOpen",
        "volume",
        "delivery",
        "OI",
        "company",
        "industry",
    ]
]
finalOHLC10yrNifty500.rename(
    columns={
        "corrClose": "close",
        "corrHigh": "high",
        "corrLow": "low",
        "corrOpen": "open",
    },
    inplace=True,
)
finalOHLC10yrNifty500.head()

,date,symbol,close,high,low,open,volume,delivery,OI,company,industry
858300,2012-03-12,3MINDIA,3931.550049,4035.000000,3925.000000,4035.000000,143.0,140.0,NaN,3M India Ltd.,CONSUMER GOODS
858301,2012-03-12,AARTIDRUGS,9.832295,9.943966,9.678085,9.784437,2043.0,1461.0,NaN,Aarti Drugs Ltd.,PHARMA
858302,2012-03-12,AARTIIND,12.014886,12.180039,11.973597,11.973597,11726.0,7709.0,NaN,Aarti Industries Ltd.,CHEMICALS
858303,2012-03-12,ABB,792.433350,818.296058,764.266555,807.809940,66703.0,18444.0,340500.0,ABB India Ltd.,INDUSTRIAL MANUFACTURING
858304,2012-03-12,ABBOTINDIA,1393.245850,1419.017053,1389.080415,1405.284630,821.0,504.0,NaN,Abbott India Ltd.,PHARMA


In [45]:
finalOHLC10yrNifty500.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [46]:
finalOHLC10yrIndex = finalOHLC10yr.loc[
    finalOHLC10yr.industry == "INDEX",
    [
        "date",
        "symbol",
        "open",
        "high",
        "low",
        "close",
        "volume",
        "delivery",
        "OI",
        "company",
        "industry",
    ],
]
finalOHLC10yrIndex.head()

,date,symbol,open,high,low,close,volume,delivery,OI,company,industry
1902681,2012-03-12,NSENIFTY,5420.10,5421.90,5327.3,5359.55,176668237.0,0.0,31408000.0,IND_NSENIFTY,INDEX
1902682,2012-03-13,NSENIFTY,5391.05,5438.65,5390.8,5429.50,179965296.0,0.0,NaN,IND_NSENIFTY,INDEX
1902683,2012-03-14,NSENIFTY,5490.55,5499.40,5437.8,5463.90,181714070.0,0.0,32741850.0,IND_NSENIFTY,INDEX
1902684,2012-03-15,NSENIFTY,5462.50,5462.50,5362.3,5380.50,160257651.0,0.0,NaN,IND_NSENIFTY,INDEX
1902685,2012-03-16,NSENIFTY,5380.35,5445.65,5305.0,5317.90,267521147.0,0.0,34712400.0,IND_NSENIFTY,INDEX


In [47]:
finalOHLC10yrIndex.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [54]:
finalOHLC10yrIndex.loc[finalOHLC10yrIndex.symbol == "NSE500", "date"].max()

datetime.date(2022, 3, 10)

In [48]:
othersImp = [
    "CRUDE_OIL",
    "GOLD",
    "HDFC_US_ADR",
    "ICICI_US_ADR",
    "INFOSYS_US_ADR",
    "USDINR",
    "DR_REDDY_US_ADR",
    "SILVER",
    "TATAMOTORS_US_ADR",
    "WIPRO_US_ADR",
]
finalOHLC10yrOther = finalOHLC10yr.loc[
    finalOHLC10yr.symbol.isin(othersImp),
    [
        "date",
        "symbol",
        "corrClose",
        "corrHigh",
        "corrLow",
        "corrOpen",
        "yfVolume",
        "delivery",
        "OI",
        "company",
        "industry",
    ],
]
finalOHLC10yrOther["company"] = "OTHERS"
finalOHLC10yrOther["industry"] = "OTHERS"
finalOHLC10yrOther.head()

,date,symbol,corrClose,corrHigh,corrLow,corrOpen,yfVolume,delivery,OI,company,industry
29435,2012-03-12,CRUDE_OIL,106.339996,107.559998,105.379997,107.500000,234727.0,NaN,NaN,OTHERS,OTHERS
29436,2012-03-12,GOLD,1699.199951,1708.099976,1694.900024,1708.099976,98.0,NaN,NaN,OTHERS,OTHERS
29437,2012-03-12,HDFC_US_ADR,15.824923,15.927259,15.745844,15.922607,1093000.0,NaN,NaN,OTHERS,OTHERS
29438,2012-03-12,ICICI_US_ADR,5.910931,5.944462,5.850257,5.912528,8518950.0,NaN,NaN,OTHERS,OTHERS
29439,2012-03-12,INFOSYS_US_ADR,5.243680,5.252786,5.171749,5.187228,13994400.0,NaN,NaN,OTHERS,OTHERS


In [49]:
finalOHLC10yrOther.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [50]:
finalOHLC10yrOther.rename(
    columns={
        "corrClose": "close",
        "corrHigh": "high",
        "corrLow": "low",
        "corrOpen": "open",
        "yfVolume": "volume",
    },
    inplace=True,
)

In [31]:
corrDf = pd.DataFrame(columns=["stock", "testStock", "corr"])
for sym in finalOHLC10yrNifty500.symbol.unique():
    industry = finalOHLC10yrNifty500.loc[
        finalOHLC10yrNifty500.symbol == sym, "industry"
    ].values[0]
    stock = finalOHLC10yrNifty500.loc[
        finalOHLC10yrNifty500.symbol == sym, ["date", "symbol", "close"]
    ].sort_values(by=["date"])
    stock["ret"] = stock["close"].pct_change()
    stockNumOfDates = stock.date.shape[0]
    for testSym in finalOHLC10yrNifty500.loc[
        (
            (finalOHLC10yrNifty500.industry == industry)
            & (finalOHLC10yrNifty500.symbol != sym)
        ),
        :,
    ]["symbol"].unique():
        testStock = finalOHLC10yrNifty500.loc[
            finalOHLC10yrNifty500.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        noOfDates = testStock.shape[0]
        if stockNumOfDates == noOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )

    for testSym in finalOHLC10yrIndex.symbol.unique():
        testStock = finalOHLC10yrIndex.loc[
            finalOHLC10yrIndex.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        testNoOfDates = testStock.shape[0]
        if stockNumOfDates == testNoOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
    for testSym in finalOHLC10yrOther.symbol.unique():
        testStock = finalOHLC10yrOther.loc[
            finalOHLC10yrOther.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        testNoOfDates = testStock.shape[0]
        if stockNumOfDates == testNoOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )

    # break

In [19]:
corrDf.sort_values(by=["corr"], ascending=False).to_csv(
    r"/Users/shubhangi/DeepDocuments/Trading/NSEData/corr.csv", index=False
)

NameError: name 'corrDf' is not defined

In [57]:
allData = finalOHLC10yrOther.append(finalOHLC10yrIndex).append(finalOHLC10yrNifty500)
allData["date"] = allData.date.apply(lambda x: x.strftime("%Y%m%d"))
allData.fillna(0, inplace=True)
allData.to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\nifty500OtherOHLC10yrs.csv",
    index=False,
)

In [58]:
allData.loc[allData.symbol == "NSE500", "date"].max()

'20220310'